In [4]:
import matplotlib
matplotlib.use('module://ipympl.backend_nbagg')

# Import os to easily join names of filepaths
import os

# Add the path of the PyOTIC Software to the system path
# Adjust this path to where the PyOTIC Software package is located'
import sys
#sys.path.insert(0, '/srv/software-repo/pyotic/')
sys.path.insert(0, '/nfs/home/tobiasj/jupyter/development/PyOTIC_github')

#Load investigator package
import pyoti
pyoti.info()

sys.path.append('/nfs/home/tobiasj/ZMBP/jupyter/development')
from tj_functions.force_extension import show_force_extension, binned_force_extension, fbnl_force_extension, plot_angle_extension

sys.path.append('/nfs/home/tobiasj/jupyter/development/tj_scripts/cn_plot_style')
import cn_plot_style as cnps

from matplotlib import pyplot as plt
from pyoti.evaluate import dna

#Create an experiment
experiment = pyoti.create_experiment()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import math
import numpy as np
import pickle

sys.path.append('..')
import unzipping_simulation as uzsi
kB = uzsi.kB

PyOTI - the investigator package of the PyOTIC software
Version: 0.12.14

The actual working path is: '/nfs/home/tobiasj/ZMBP/jupyter/development/unzipping_simulation/notebooks'


In [2]:
# Convert old simulations to new ones
_E_ext_ssDNA = uzsi.E_ext_ssDNA
EXT_DSDNA = uzsi.EXT_DSDNA
_ext_dsDNA_wlc = uzsi.ext_dsDNA_wlc
unboundfunction = uzsi.unboundfunction

with open('results','rb') as f:
    simulations = pickle.load(f)

for old_key in list(simulations.keys()):
    del simulations[old_key]['E_ext_ssDNA']
    del simulations[old_key]['ext_dsDNA_wlc']
    s = simulations[old_key]['settings']
    new_key = uzsi.get_key(
        s['x0_min'],
        s['x0_max'],
        s['y0'],
        s['h0'],
        s['resolution'],
        s['bases'],
        s['nbs'],
        s['nbp'],
        s['nbs_loop'],
        s['radius'],
        s['angles_r0'],
        s['kappa'],
        s['k_rot'],
        s['S'],
        s['L_p_ssDNA'],
        s['z'],
        s['pitch'],
        s['L_p_dsDNA'],
        s['NNBP'],
        s['c'],
        s['e_loop'],
        s['T'],
        s['boltzmann_factor'])
    simulations[new_key] = simulations[old_key]
    simulations.pop(old_key)

#with open('simulations.p','wb') as f:
#    pickle.dump(simulations, f)

In [28]:
# Load simulations
with open('simulations.p','rb') as f:
    simulations = pickle.load(f)

[[key, simulation['settings']['h0'], simulation['settings']['kappa']] for key, simulation in simulations.items()]

[['1544d1fa0d5e6c3c86854bcff2cf5548', 1.57e-07, array([0.00068, 0.00014])]]

In [29]:
simulations['1544d1fa0d5e6c3c86854bcff2cf5548']['settings']

{'x0_min': 1e-07,
 'x0_max': 1.7e-06,
 'y0': 0.0,
 'h0': 1.57e-07,
 'resolution': 2e-09,
 'bases': 'GATACGTTCTTACCCATACTCCACCGTTGCTGTGCCAACACATTGCgactatcgcaccatcagccagaaaaccgaattttgctgggtgggctaacgatatccgcctgatgcgtgaacgtgacggacgtaaccaccgcgacatgtgtgtgctgttccgctgggcatgccaggacaacttctggtccggtaacgtgctgagcccggccaaactccgcgataagtggacccaactcgaaatcaaccgtaacaagcaacaggcaggcgtgacagccagcaaaccaaaactcgacctgacaaacacagactggatttacggggtggatctatgaaaaacatcgccgcacagatggttaactttgaccgtgagcagatgcgtcggatcgccaacaacatgccggaacagtacgacgaaaagccgcaggtacagcaggtagcgcagatcatcaacggtgtgttcagccagttactggcaactttcccggcgagcctggctaaccgtgaccagaacgaagtgaacgaaatccgtcgccagtgggttctggcttttcgggaaaacgggatcaccacgatggaacaggttaacgcaggaatgcgcgtagcccgtcggcagaatcgaccatttctgccatcacccgggcagtttgttgcatggtgccgggaagaagcatccgttaccgccggactgccaaacgtcagcgagctggttgatatggtttacgagtattgccggaagcgaggcctgtatccggatgcggagtcttatccgtggaaatcaaacgcgcactGCAATGCAAGCTACTGCCGGTCGTAT',
 'nbs': 10,
 'nbp': 1399,
 'nbs_loop': 10,
 'radius': 5.01e-07,
 'angles_r0': None,
 '

In [30]:
with open('result.p','wb') as f:
    pickle.dump(simulations['1544d1fa0d5e6c3c86854bcff2cf5548'], f)

In [31]:
# Choose the path, were the experiment should be created (or opened from)
#
# datadir: The path to where the experiment (and the data) are located
# datafile: The name of the file that contains the data. Here it is only used to generate dbfile.
#           The data is loaded further down upon creation of a Record.
# dbfile: The name of the database file the experiment is saved to (or loaded from).
#datadir =  '../../../../data/ASWAD/unzipping/2015-03-12 - unzipping/data/'
datadir = '/nfs/home/tobiasj/data/ASWAD/unzipping/2018-05-16 - unzipping N50TT20AA0.1BSA/data/'
#datadir =  '../../../../data/ASWAD/unzipping/2017-08-01_03 - unzipping and biotin concentration/0.1 percent - unzipping and jumping in trap, l_2, l_4 plate/data/'
#datafile = 'B09.bin'
datafile = 'B01.bin'

# Select the unzipping number(s)
I = [205]
sortcolumn = 0  # 0: time, 1: extension
resolution = 1000


# For the name of the experiment, exchange the extension '.bin' with '.fs'
dbfile = os.path.join(datadir, datafile.replace('.bin', '.fs'))

# Open the experiment dbfile
experiment.open(dbfile)
results_region = experiment.view('results')

# Choose resolution for tether object
resolution_tether = results_region.samplingrate # points/s resolution
resolution_sf = 1000
tether = pyoti.create_tether(region=results_region, resolution=resolution_tether, resolution_sf=resolution_sf)

# Get force extensions from unzipping number(s)
x = []
F = []
FXYZ = []
DISTXYZ = []
for i in I:
    # Get binned force extension
    bin_edges, bin_centers, bin_widths, bin_means, bin_stds, bin_Ns \
        = binned_force_extension(tether, i, resolution=resolution, sortcolumn=sortcolumn, extra_traces=['psdXYZ', 'positionXYZ'])
    _x = bin_means[0][:,1]
    _y = bin_means[0][:,2]
    x.append(_x)
    F.append(_y)

    # Select corresponding bin_means and assign to easy to remember variable names
    data = bin_means[0][:,3:]
    psdXYZ = data[:, 0:3]
    positionXYZ = data[:, 3:6]
    positionXY = positionXYZ[:, 0:2]
    positionZ = data[:, [5]]

    # Calculate 3D force
    displacementXYZ \
        = tether.calibration.displacement(psdXYZ, positionZ=positionZ)
    distXYZ = pyoti.evaluate.tether.distanceXYZ(tether.calibration, psdXYZ, positionXYZ)  # m
    DISTXYZ.append(distXYZ)
    dist = pyoti.evaluate.tether.distance(distXYZ, positionXY)
    forceXY_Z = tether.calibration.force(displacementXYZ, positionZ=positionZ)
    fXYZ = pyoti.evaluate.tether.forceXYZ(tether.calibration, psdXYZ, positionZ)  # N
    fXYZ[:,2] *= 0.75
    FXYZ.append(fXYZ)

experiment.abort()
experiment.close()

Open experiment file:
   '/nfs/home/tobiasj/data/ASWAD/unzipping/2018-05-16 - unzipping N50TT20AA0.1BSA/data/B01.fs'
Cannot open the experiment file!
 The database is locked by another process or user.
 The original error message is: 'Couldn't lock '/nfs/home/tobiasj/data/ASWAD/unzipping/2018-05-16 - unzipping N50TT20AA0.1BSA/data/B01.fs.lock''
Experiment is closed!


AttributeError: 'NoneType' object has no attribute 'samplingrate'

In [11]:
plt.close('all')
for key, simulation in simulations.items():
#for simulation in [simulations['592d5ab2f569ae1753ab5716385abfd2']]:
    # scale the measured data
    shift_x = 20.0e-9  # m
    scale_x = 1.0
    shift_y = 0.0e-12  # N
    scale_y = 1.0

    with cnps.cn_plot(context='presentation', fig_scale=1.0, dark=False) as cnp:
        for i in range(len(I)):
            fig, axes = uzsi.plot_simulated_force_extension(simulation,
                                                            x[i] * scale_x + shift_x,
                                                            F[i] * scale_y + shift_y,
                                                            np.abs(FXYZ[i]) * scale_y + shift_y,
                                                            ylim=(-1, 22))
            fig.suptitle('{} {}'.format(key, round(simulation['settings']['k_rot'][0]*180/math.pi*1e12)))
            fig.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [10]:
#x[0], FXYZ[0]
i = 0
angles_E = np.array([uzsi.cart2sph(*point)[1:] for point in DISTXYZ[i]])*180/math.pi
#angles_E[:,1] -= 90
angles_F = np.array([uzsi.cart2sph(*point)[1:] for point in FXYZ[i]])*180/math.pi
#angles_F[:,1] -= 90
angles_diff = angles_E - angles_F
plt.ion()
fig, ax = plt.subplots()
ax.plot(x[i], angles_E[:,0])
ax.plot(x[i], angles_F[:,0])
ax.plot(x[i], angles_diff[:,0])
ax.set_xlabel('Apparent extension (nm)')
ax.set_ylabel('Theta (°)')
ax.set_title('Theta')
fig, ax = plt.subplots()
ax.plot(x[i], angles_E[:,1])
ax.plot(x[i], angles_F[:,1])
#ax.plot(x[i], angles_diff[:,1])
ax.set_xlabel('Apparent extension (nm)')
ax.set_ylabel('Phi (°)')
ax.set_title('Phi')

/usr/local/lib/python3.7/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

/usr/local/lib/python3.7/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

Text(0.5, 1.0, 'Phi')